In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#from sklearn.linear_model import LogisticRegression
#from sklearn.neural_network import MLPClassifier


In [2]:
var_30 = pd.read_csv('top_30_wrapper.csv')
all_df = pd.read_csv('all_df_var.csv')

In [3]:
var_model = var_30['field'].to_list() + ['Fraud'] + ['Date']
pre_df = all_df[var_model]

In [4]:
train = pre_df[pre_df.Date <= '2010-08-31']
oot = pre_df[pre_df.Date > '2010-08-31']

In [5]:
FDR3 = pd.DataFrame([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], columns=['train'])
FDR3['test'] = 0
FDR3['oot'] = 0
FDR3

,train,test,oot
0,0,0,0
1,1,0,0
2,2,0,0
3,3,0,0
4,4,0,0
5,5,0,0
6,6,0,0
7,7,0,0
8,8,0,0
9,9,0,0


In [6]:
var_model_final = var_30['field'].to_list()# + ['Fraud']
X = train[var_model_final]
y = train['Fraud'].values
X_oot = oot[var_model_final]
y_oot = oot['Fraud'].values

In [7]:
scaler = StandardScaler()    
X = scaler.fit_transform(X)
X_oot = scaler.fit_transform(X_oot)

### Logistic regression

In [8]:
evaluation = pd.DataFrame([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], columns=['Logistic Regression'])
evaluation['max_iter'] = [15000, 15000, 10000, 10000, 5000, 5000, 2000, 2000, 2000, 1000]
evaluation['C'] = [1, 1, 2, 1, 1.5, 1.5, 1.5, 0.5, 0.5, 0.5]
evaluation['Class_weight'] = 0.0107
evaluation['solver'] = ['lbfgs', 'liblinear', 'lbfgs', 'liblinear', 'lbfgs', 'liblinear', 'lbfgs', 'liblinear', 'liblinear', 'lbfgs']
evaluation['train'] = 0
evaluation['test'] = 0
evaluation['oot'] = 0
evaluation

,Logistic Regression,max_iter,C,Class_weight,solver,train,test,oot
0,1,15000,1.0,0.0107,lbfgs,0,0,0
1,2,15000,1.0,0.0107,liblinear,0,0,0
2,3,10000,2.0,0.0107,lbfgs,0,0,0
3,4,10000,1.0,0.0107,liblinear,0,0,0
4,5,5000,1.5,0.0107,lbfgs,0,0,0
5,6,5000,1.5,0.0107,liblinear,0,0,0
6,7,2000,1.5,0.0107,lbfgs,0,0,0
7,8,2000,0.5,0.0107,liblinear,0,0,0
8,9,2000,0.5,0.0107,liblinear,0,0,0
9,10,1000,0.5,0.0107,lbfgs,0,0,0


In [9]:
for idx in evaluation.index:
    max_iter_it = evaluation.loc[idx, 'max_iter']
    C_it = evaluation.loc[idx, 'C']
    weight_it = evaluation.loc[idx, 'Class_weight']
    solve_it = evaluation.loc[idx, 'solver']

    for niter in range(10):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

        X_oot_eval = pd.DataFrame(X_oot.copy())
        X_train_eval = pd.DataFrame(X_train.copy())
        X_test_eval = pd.DataFrame(X_test.copy())

        clf = LogisticRegression(max_iter=max_iter_it, 
                                 C=C_it, 
                                 class_weight={1:y_train.mean()}, 
                                 solver=solve_it)
        clf.fit(X_train, y_train)

        predictions = clf.predict_proba(X_train)[:, 1]

        X_train_eval['pred'] = predictions
        X_train_eval['Fraud'] = y_train
        topRows = int(round(X_train_eval.shape[0]*0.03))
        temp = X_train_eval.sort_values('pred', ascending=False).head(topRows)
        needed = temp.loc[:, 'Fraud']
        FDR3.loc[niter, 'train'] = sum(needed)/sum(X_train_eval.loc[:, 'Fraud'])


        predictions = clf.predict_proba(X_test)[:, 1]

        X_test_eval['pred'] = predictions
        X_test_eval['Fraud'] = y_test
        topRows = int(round(X_test_eval.shape[0]*0.03))
        temp = X_test_eval.sort_values('pred', ascending=False).head(topRows)
        needed = temp.loc[:, 'Fraud']
        FDR3.loc[niter, 'test'] = sum(needed)/sum(X_test_eval.loc[:, 'Fraud'])

        predictions = clf.predict_proba(X_oot)[:, 1]

        X_oot_eval['pred'] = predictions
        X_oot_eval['Fraud'] = y_oot
        topRows = int(round(X_oot_eval.shape[0]*0.03))
        temp = X_oot_eval.sort_values('pred', ascending=False).head(topRows)
        needed = temp.loc[:, 'Fraud']
        FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot_eval.loc[:, 'Fraud'])

        #print(niter, FDR3.loc[niter, 'train'], FDR3.loc[niter, 'test'], FDR3.loc[niter, 'oot'])
        
    evaluation.loc[idx, 'train'] = FDR3.mean()['train']
    evaluation.loc[idx, 'test'] = FDR3.mean()['test']
    evaluation.loc[idx, 'oot'] = FDR3.mean()['oot']

In [10]:
logit = evaluation.copy()
logit.to_csv('logit.csv', index=False)
logit

,Logistic Regression,max_iter,C,Class_weight,solver,train,test,oot
0,1,15000,1.0,0.0107,lbfgs,0.689686,0.690679,0.540730
1,2,15000,1.0,0.0107,liblinear,0.701565,0.712635,0.548315
2,3,10000,2.0,0.0107,lbfgs,0.699672,0.700382,0.546629
3,4,10000,1.0,0.0107,liblinear,0.696527,0.690951,0.549719
4,5,5000,1.5,0.0107,lbfgs,0.695985,0.712502,0.571348
5,6,5000,1.5,0.0107,liblinear,0.699196,0.683139,0.540169
6,7,2000,1.5,0.0107,lbfgs,0.702908,0.696351,0.541292
7,8,2000,0.5,0.0107,liblinear,0.686484,0.688254,0.521348
8,9,2000,0.5,0.0107,liblinear,0.686887,0.674671,0.537360
9,10,1000,0.5,0.0107,lbfgs,0.683483,0.678804,0.507303


### MLP Classifier

In [14]:
nr_hnodes = range(1, 5)
hls = list(zip(nr_hnodes)) + list(zip(nr_hnodes, nr_hnodes))

evaluation = pd.DataFrame([1, 2, 3, 4, 5, 6, 7, 8], columns=['MLP Classifier'])
evaluation['max_iter'] = 10000
evaluation['alpha'] = [0.0001, 0.0001, 0.001, 0.001, 0.01, 0.01, 0.01, 0.05]
evaluation['activation']=['relu', 'tanh','relu', 'logistic', 'tanh', 'relu', 'tanh','relu']
evaluation['solver'] = ['lbfgs', 'lbfgs', 'lbfgs', 'lbfgs', 'adam', 'adam', 'adam', 'adam']
evaluation['hidden_layer_sizes'] = hls
evaluation['train'] = 0
evaluation['test'] = 0
evaluation['oot'] = 0
evaluation

,MLP Classifier,max_iter,alpha,activation,solver,hidden_layer_sizes,train,test,oot
0,1,10000,0.0001,relu,lbfgs,"(1,)",0,0,0
1,2,10000,0.0001,tanh,lbfgs,"(2,)",0,0,0
2,3,10000,0.0010,relu,lbfgs,"(3,)",0,0,0
3,4,10000,0.0010,logistic,lbfgs,"(4,)",0,0,0
4,5,10000,0.0100,tanh,adam,"(1, 1)",0,0,0
5,6,10000,0.0100,relu,adam,"(2, 2)",0,0,0
6,7,10000,0.0100,tanh,adam,"(3, 3)",0,0,0
7,8,10000,0.0500,relu,adam,"(4, 4)",0,0,0


In [15]:
for idx in evaluation.index:
    max_iter_it = evaluation.loc[idx, 'max_iter']
    alpha_it = evaluation.loc[idx, 'alpha']
    solve_it = evaluation.loc[idx, 'solver']
    act_it = evaluation.loc[idx, 'activation']
    hid_it = evaluation.loc[idx, 'hidden_layer_sizes']
    for niter in range(10):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

        X_oot_eval = pd.DataFrame(X_oot.copy())
        X_train_eval = pd.DataFrame(X_train.copy())
        X_test_eval = pd.DataFrame(X_test.copy())


        mlp = MLPClassifier(activation=act_it,
                       solver=solve_it,
                       alpha=alpha_it,
                       hidden_layer_sizes=hid_it,
                       max_iter=max_iter_it,
                      )#.fit(X_train, y_train)
        mlp.fit(X_train, y_train)

        predictions = mlp.predict_proba(X_train)[:, 1]

        X_train_eval['pred'] = predictions
        X_train_eval['Fraud'] = y_train
        topRows = int(round(X_train_eval.shape[0]*0.03))
        temp = X_train_eval.sort_values('pred', ascending=False).head(topRows)
        needed = temp.loc[:, 'Fraud']
        FDR3.loc[niter, 'train'] = sum(needed)/sum(X_train_eval.loc[:, 'Fraud'])


        predictions = mlp.predict_proba(X_test)[:, 1]

        X_test_eval['pred'] = predictions
        X_test_eval['Fraud'] = y_test
        topRows = int(round(X_test_eval.shape[0]*0.03))
        temp = X_test_eval.sort_values('pred', ascending=False).head(topRows)
        needed = temp.loc[:, 'Fraud']
        FDR3.loc[niter, 'test'] = sum(needed)/sum(X_test_eval.loc[:, 'Fraud'])

        predictions = mlp.predict_proba(X_oot)[:, 1]

        X_oot_eval['pred'] = predictions
        X_oot_eval['Fraud'] = y_oot
        topRows = int(round(X_oot_eval.shape[0]*0.03))
        temp = X_oot_eval.sort_values('pred', ascending=False).head(topRows)
        needed = temp.loc[:, 'Fraud']
        FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot_eval.loc[:, 'Fraud'])

        #print(niter, FDR3.loc[niter, 'train'], FDR3.loc[niter, 'test'], FDR3.loc[niter, 'oot'])
        
    evaluation.loc[idx, 'train'] = FDR3.mean()['train']
    evaluation.loc[idx, 'test'] = FDR3.mean()['test']
    evaluation.loc[idx, 'oot'] = FDR3.mean()['oot']
    print(idx)

0
1
2
3
4
5
6
7


In [17]:
mlp_result = evaluation.copy()
mlp_result.to_csv('mlp_result.csv', index=False)
mlp_result

,MLP Classifier,max_iter,alpha,activation,solver,hidden_layer_sizes,train,test,oot
0,1,10000,0.0001,relu,lbfgs,"(1,)",0.767268,0.765371,0.592135
1,2,10000,0.0001,tanh,lbfgs,"(2,)",0.838929,0.829529,0.491854
2,3,10000,0.0010,relu,lbfgs,"(3,)",0.882396,0.846827,0.505337
3,4,10000,0.0010,logistic,lbfgs,"(4,)",0.904238,0.865718,0.511517
4,5,10000,0.0100,tanh,adam,"(1, 1)",0.740667,0.726544,0.570506
5,6,10000,0.0100,relu,adam,"(2, 2)",0.796663,0.793220,0.504775
6,7,10000,0.0100,tanh,adam,"(3, 3)",0.891064,0.865567,0.512079
7,8,10000,0.0500,relu,adam,"(4, 4)",0.862930,0.849362,0.565730


### xgboost

In [8]:
evaluation = pd.DataFrame([1, 2, 3, 4, 5, 6, 7], columns=['xgboost'])
evaluation['objective'] = 'binary:logistic'
evaluation['colsample_bytree'] = [0.3, 0.3, 0.7, 0.7, 0.7, 1, 1]
evaluation['n_estimators']= [50, 100, 50, 100, 150, 100, 150]
evaluation['max_depth']=[2, 5, 7, 10, 2, 5, 7]
evaluation['eta'] = [0.01,0.1,0.5,0.9, 0.01, 0.5, 0.1]
evaluation['train'] = 0
evaluation['test'] = 0
evaluation['oot'] = 0
evaluation

,xgboost,objective,colsample_bytree,n_estimators,max_depth,eta,train,test,oot
0,1,binary:logistic,0.3,50,2,0.01,0,0,0
1,2,binary:logistic,0.3,100,5,0.10,0,0,0
2,3,binary:logistic,0.7,50,7,0.50,0,0,0
3,4,binary:logistic,0.7,100,10,0.90,0,0,0
4,5,binary:logistic,0.7,150,2,0.01,0,0,0
5,6,binary:logistic,1.0,100,5,0.50,0,0,0
6,7,binary:logistic,1.0,150,7,0.10,0,0,0


In [9]:
import xgboost as xgb

for idx in evaluation.index:
    obj_it = evaluation.loc[idx, 'objective']
    col_it = evaluation.loc[idx, 'colsample_bytree']
    estim_it = evaluation.loc[idx, 'n_estimators']
    dep_it = evaluation.loc[idx, 'max_depth']
    eta_it = evaluation.loc[idx, 'eta']
    
    for niter in range(10):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

        X_oot_eval = pd.DataFrame(X_oot.copy())
        X_train_eval = pd.DataFrame(X_train.copy())
        X_test_eval = pd.DataFrame(X_test.copy())


        gbm = xgb.XGBClassifier(use_label_encoder = False, 
                               objective=obj_it,
                               colsample_bytree=col_it, 
                               n_estimators=estim_it, 
                               max_depth=dep_it, 
                               eta=eta_it
                               )
    
        
        gbm.fit(X_train, y_train)

        predictions = gbm.predict_proba(X_train)[:, 1]

        X_train_eval['pred'] = predictions
        X_train_eval['Fraud'] = y_train
        topRows = int(round(X_train_eval.shape[0]*0.03))
        temp = X_train_eval.sort_values('pred', ascending=False).head(topRows)
        needed = temp.loc[:, 'Fraud']
        FDR3.loc[niter, 'train'] = sum(needed)/sum(X_train_eval.loc[:, 'Fraud'])


        predictions = gbm.predict_proba(X_test)[:, 1]

        X_test_eval['pred'] = predictions
        X_test_eval['Fraud'] = y_test
        topRows = int(round(X_test_eval.shape[0]*0.03))
        temp = X_test_eval.sort_values('pred', ascending=False).head(topRows)
        needed = temp.loc[:, 'Fraud']
        FDR3.loc[niter, 'test'] = sum(needed)/sum(X_test_eval.loc[:, 'Fraud'])

        predictions = gbm.predict_proba(X_oot)[:, 1]

        X_oot_eval['pred'] = predictions
        X_oot_eval['Fraud'] = y_oot
        topRows = int(round(X_oot_eval.shape[0]*0.03))
        temp = X_oot_eval.sort_values('pred', ascending=False).head(topRows)
        needed = temp.loc[:, 'Fraud']
        FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot_eval.loc[:, 'Fraud'])

        #print(niter, FDR3.loc[niter, 'train'], FDR3.loc[niter, 'test'], FDR3.loc[niter, 'oot'])
    print(idx)    
    evaluation.loc[idx, 'train'] = FDR3.mean()['train']
    evaluation.loc[idx, 'test'] = FDR3.mean()['test']
    evaluation.loc[idx, 'oot'] = FDR3.mean()['oot']
    print(idx)

[21:36:55] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:39:09] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:41:06] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:43:09] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

KeyboardInterrupt: 

In [10]:
xgb_result = evaluation.copy()
xgb_result.to_csv('xgb_result.csv', index=False)
xgb_result

,xgboost,objective,colsample_bytree,n_estimators,max_depth,eta,train,test,oot
0,1,binary:logistic,0.3,50,2,0.01,0.707405,0.704800,0.570225
1,2,binary:logistic,0.3,100,5,0.10,0.995943,0.980536,0.436798
2,3,binary:logistic,0.7,50,7,0.50,0.000000,0.000000,0.000000
3,4,binary:logistic,0.7,100,10,0.90,0.000000,0.000000,0.000000
4,5,binary:logistic,0.7,150,2,0.01,0.000000,0.000000,0.000000
5,6,binary:logistic,1.0,100,5,0.50,0.000000,0.000000,0.000000
6,7,binary:logistic,1.0,150,7,0.10,0.000000,0.000000,0.000000


### random forest

In [14]:
evaluation = pd.DataFrame([1, 2, 3, 4, 5, 6], columns=['ramdom forest'])

evaluation['max_features'] = ['auto', 'sqrt', 'auto', 'sqrt', 'auto','sqrt']
evaluation['n_estimators']= [50, 100, 50, 100, 100, 100]
evaluation['max_depth']=[2, 5, 7, 10, 5, 7]
evaluation['min_samples_split'] = [2, 5, 2, 5, 3, 5]
evaluation['min_samples_leaf'] = [1, 2, 4, 1, 2, 4]
evaluation['bootstrap'] = [True, False, True, True, True, False]

evaluation['train'] = 0
evaluation['test'] = 0
evaluation['oot'] = 0
evaluation

,ramdom forest,max_features,n_estimators,max_depth,min_samples_split,min_samples_leaf,bootstrap,train,test,oot
0,1,auto,50,2,2,1,True,0,0,0
1,2,sqrt,100,5,5,2,False,0,0,0
2,3,auto,50,7,2,4,True,0,0,0
3,4,sqrt,100,10,5,1,True,0,0,0
4,5,auto,100,5,3,2,True,0,0,0
5,6,sqrt,100,7,5,4,False,0,0,0


In [15]:
from sklearn.ensemble import RandomForestClassifier

for idx in evaluation.index:
    maxfet_it = evaluation.loc[idx, 'max_features']
    samsplt_it = evaluation.loc[idx, 'min_samples_split']
    leaf_it = evaluation.loc[idx, 'min_samples_leaf']
    estim_it = evaluation.loc[idx, 'n_estimators']
    dep_it = evaluation.loc[idx, 'max_depth']
    boot_it = evaluation.loc[idx, 'bootstrap']
    
    for niter in range(10):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

        X_oot_eval = pd.DataFrame(X_oot.copy())
        X_train_eval = pd.DataFrame(X_train.copy())
        X_test_eval = pd.DataFrame(X_test.copy())
        
        rfc = RandomForestClassifier(n_estimators=estim_it, 
                                    max_features = maxfet_it, 
                                    min_samples_split=samsplt_it, 
                                    min_samples_leaf=leaf_it, 
                                    max_depth=dep_it, 
                                    bootstrap=boot_it
                                    )

    
        
        rfc.fit(X_train, y_train)

        predictions = rfc.predict_proba(X_train)[:, 1]

        X_train_eval['pred'] = predictions
        X_train_eval['Fraud'] = y_train
        topRows = int(round(X_train_eval.shape[0]*0.03))
        temp = X_train_eval.sort_values('pred', ascending=False).head(topRows)
        needed = temp.loc[:, 'Fraud']
        FDR3.loc[niter, 'train'] = sum(needed)/sum(X_train_eval.loc[:, 'Fraud'])


        predictions = rfc.predict_proba(X_test)[:, 1]

        X_test_eval['pred'] = predictions
        X_test_eval['Fraud'] = y_test
        topRows = int(round(X_test_eval.shape[0]*0.03))
        temp = X_test_eval.sort_values('pred', ascending=False).head(topRows)
        needed = temp.loc[:, 'Fraud']
        FDR3.loc[niter, 'test'] = sum(needed)/sum(X_test_eval.loc[:, 'Fraud'])

        predictions = rfc.predict_proba(X_oot)[:, 1]

        X_oot_eval['pred'] = predictions
        X_oot_eval['Fraud'] = y_oot
        topRows = int(round(X_oot_eval.shape[0]*0.03))
        temp = X_oot_eval.sort_values('pred', ascending=False).head(topRows)
        needed = temp.loc[:, 'Fraud']
        FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot_eval.loc[:, 'Fraud'])

        #print(niter, FDR3.loc[niter, 'train'], FDR3.loc[niter, 'test'], FDR3.loc[niter, 'oot'])
        
    evaluation.loc[idx, 'train'] = FDR3.mean()['train']
    evaluation.loc[idx, 'test'] = FDR3.mean()['test']
    evaluation.loc[idx, 'oot'] = FDR3.mean()['oot']
    print(idx)

0
1
2
3
4
5


In [16]:
rfm_result = evaluation.copy()
rfm_result.to_csv('rfm_result.csv', index=False)
rfm_result

,ramdom forest,max_features,n_estimators,max_depth,min_samples_split,min_samples_leaf,bootstrap,train,test,oot
0,1,auto,50,2,2,1,True,0.729084,0.708121,0.574157
1,2,sqrt,100,5,5,2,False,0.890272,0.885168,0.503933
2,3,auto,50,7,2,4,True,0.925677,0.912062,0.541011
3,4,sqrt,100,10,5,1,True,0.980457,0.950629,0.540449
4,5,auto,100,5,3,2,True,0.881112,0.864834,0.501124
5,6,sqrt,100,7,5,4,False,0.921347,0.902321,0.529775
